In [36]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '../../'))
sys.path.append(parent_dir)
from em_utils import * # import the utils

In [37]:
# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/merf_dfs/"
os.makedirs(df_dir, exist_ok=True)

print("---------- Read data ----------")
input_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/march_20/"
long = read_data(input_dir,"all_processing_long.csv")
delta = read_data(input_dir, "all_processing_0_6m.csv")

---------- Read data ----------


In [38]:
columns_to_drop = ['Peptide_YY', 'Ghrelin', 'Leptin', 'completer', 'consent', 'Hemoglobin_A1C', 'record_id']
long = long.drop([col for col in columns_to_drop if col in long.columns], axis=1)
print(long.columns.to_list)
#Make time numeric 
time_mapping = {
    "BL": 0,
    "6m": 6,
    "12m": 12
}
long['time'] = long['time'].replace(time_mapping)
print(long['time'])

<bound method IndexOpsMixin.tolist of Index(['subject_id', 'time', 'outcome_BMI_fnl', 'randomized_group',
       'cohort_number', 'sex', 'race', 'age', 'bmi_prs', 'Glucose',
       ...
       'Isovalerate..3.Methylbutanoate', 'phenylacetate', 'Succinate',
       'Isobutyrate..2.Methylpropanoate', 'Isocaproate', 'Water', 'Hydrogen',
       'Propionate', 'Acetate', 'Carbon.dioxide'],
      dtype='object', length=552)>
0       0
1       0
2       0
3       0
4       0
       ..
350    12
351    12
352    12
353    12
354    12
Name: time, Length: 355, dtype: int64


/var/folders/4l/kqxc4rwn4lj10fzv9hw4xxsr0000gn/T/ipykernel_26667/1123710259.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  long['time'] = long['time'].replace(time_mapping)


In [39]:
BMI_VAR, ID_VAR, TIME_VAR, DATA = 'outcome_BMI_fnl', 'subject_id', 'time', long
# Specify columns for long
basic = [BMI_VAR, ID_VAR, TIME_VAR, 'age','sex']
meta_keep = [BMI_VAR, ID_VAR, TIME_VAR, 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'Glucose', 'HDL_Total_Direct_lipid', 'HOMA_IR', 'Insulin_endo', 'LDL_Calculated', 'Triglyceride_lipid']
only_grs = [BMI_VAR, ID_VAR, TIME_VAR, 'bmi_prs']
only_taxa = [BMI_VAR, ID_VAR, TIME_VAR] + [col for col in DATA.columns if col.startswith("g__")]
proton_column = DATA.columns.get_loc("proton")
carbon_dioxide_column = DATA.columns.get_loc("Carbon.dioxide")
only_micom = [BMI_VAR, ID_VAR, TIME_VAR] + [col for col in DATA.columns if col in DATA.columns[proton_column:carbon_dioxide_column+1]]
# Create a set of all columns to EXCLUDE for pathway
exclude_columns = set(meta_keep + only_grs + only_taxa + only_micom)
only_pathway = [BMI_VAR, ID_VAR, TIME_VAR] + [col for col in DATA.columns if col not in exclude_columns]


Set up MERF 

In [40]:
# test sample names
test_names = ["ABR-079", "AGA-071", "AHE-055", "ALI-121", "ALO-163", "AMA-031", "ASO-013", "AWI-167", "BMO-164", "CWA-183", "DSC-024", "EBE-130", "EHI-177", "EJO-092", "GFU-188", "HGI-010", "JCA-109", "JGO-100",
    "KBU-085", "KCE-034", "KHE-170", "LDO-148", "LST-186", "LZD-142", "MAR-119", "MCA-088", "MJA-153", "MWE-112", "NPO-149", "RAE-114", "SBO-020", "SEG-080", "SKA-195", "SLO-178", "SSH-028", "TDU-086","TFA-016", "VCA-041"]
# train sample names
train_names = ["AAL-144", "ACO-053", "ADA-105", "AKE-009", "AKI-011", "AKO-139", "AMC-155", "AME-128", "AME-157", "ATA-129", "AWA-052", "AWA-083", "BAN-193", "BHO-014", "BIN-201", "BKN-104", "BMI-156", "BSA-174", "CAM-057", "CCO-189",
    "CED-026", "CEL-073", "CGA-134", "CIS-077", "CKR-078", "CLE-049", "COW-066", "CRO-108", "CWA-161", "EBE-051", "EKA-135", "EKR-045", "ELA-159", "EPO-182", "EVO-184", "FWI-098", "GHA-035", "HDE-154", "IBE-120", "JDI-140", "JER-110", "JFU-027", "JJO-093", "JKN-127", "JPO-022", "JUG-116", "JUT-032", "JVE-126", "KAN-138", "KBR-162", "KEL-185", "KEL-199", "KGI-029", "KHU-196", "KPA-042", "KRI-072", "KVA-038", "KWA-122", "KWA-141", "LBL-047", "LBU-015", "LEL-147", "LFI-003", "LJA-101", "LMC-111", "LPF-198", "LVA-017", "MBA-187", "MCW-065", "MDI-107", "MES-068", "MFB-118", "MGA-076", "MHO-117", "MKE-192", "MMA-036", "MRT-179", "MSH-091", "MST-039", "MWE-143",
    "MWO-133", "MWY-152", "NAR-099", "NBI-048", "NBI-069", "NCO-171", "NDI-067", "NEL-094", "NKA-090", "NMO-151", "NTA-021", "PBE-123", "QNG-166", "RAF-125", "RAM-050", "RHP-023", "RLA-132", "ROL-006", "SAB-160", "SCA-043", "SCR-061", "SDA-150", "SGA-062", "SKA-087", "SRO-194", "TBU-115", "TFA-172", "TRO-113", "TSH-146", "TSL-056", "WPE-005", "YOR-103", "YSU-097", "ZVU-096"]
print("Length of test names:", len(test_names))
print("Length of train names:", len(train_names))

Length of test names: 38
Length of train names: 114


In [41]:
column_sets = {
    "basic": basic,
    "meta_keep": meta_keep,
    "only_grs": only_grs, 
    "only_taxa": only_taxa,
    "only_micom": only_micom,
    "only_pathway": only_pathway
}
# Loop through each column set and create DATA
for key, columns in column_sets.items():
    DATA = long[columns]
    print(f"Subset for {key} created with shape: {DATA.shape}")
    OUT = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/finetuning_csvs/{key}_long.csv"
    output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/final_merf_plots/"
    r2_out =  f"{key}_long_r2.pdf"
    r2_adj_out = f"{key}_long_r2_adj.pdf"
    feature_imp_out = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/final_merf_plots/{key}_long_ft_imp.pdf"
    results_filename = f"/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/march30/final_merf_dfs/{key}_long.csv"

# Create train and test sets for the current subset
train_set = DATA[DATA[ID_VAR].isin(train_names)]  # Select rows where ID_VAR is in train_names
test_set = DATA[DATA[ID_VAR].isin(test_names)]    # Select rows where ID_VAR is in test_names

# Ensure clusters_train and clusters_test are pandas Series
clusters_train = pd.Series(train_set[ID_VAR])  # Convert to pandas Series
clusters_test = pd.Series(test_set[ID_VAR])    # Convert to pandas Series

# Proceed with MERF analysis for the current subset
print("---------- Select predictors for training set ----------")
X = train_set.drop([BMI_VAR, ID_VAR], axis=1)
Y = train_set[BMI_VAR].to_numpy()  # Convert Y to numeric array
Z = np.ones((train_set.shape[0], 1))  # Create random effects matrix with ones

# Check lengths before fitting
print(f"Length of X: {len(X)}, Length of clusters_train: {len(clusters_train)}, Length of Y: {len(Y)}")
assert len(X) == len(clusters_train), "Length of X does not match length of clusters_train"
assert len(X) == len(Y), "Length of X does not match length of Y"
print("Final columns after drop:", X.columns.to_list())
print("X train values:", train_set[BMI_VAR])

print("---------- Select predictors for test set ----------")
X_new = test_set.drop([BMI_VAR, ID_VAR], axis=1)
X_new = X_new[X.columns]  # Reorder and select columns to match training set
X_new = X_new.astype(X.dtypes)  # Ensure data types match
Y_new = test_set[BMI_VAR].to_numpy()  # Convert Y to numeric array
clusters_new = pd.Series(test_set[ID_VAR])  # Convert to pandas Series
Z_new = np.ones((len(X_new), 1))  # Create random effects matrix with ones
time_new = test_set[TIME_VAR].astype(float).to_numpy()  # Convert time values to numeric arrayo numeric array

# Hyperparameters to tune
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None],
    'min_samples_split': [0.05, 0.1, 0.15],
    'max_iter': [2, 10],
    'n_splits': [3, 5, 10] #cross-validation
}
# Create training features
# X = train_set.drop(columns=columns_to_drop, errors='ignore')
y = train_set[[BMI_VAR]]
y = y[BMI_VAR].to_numpy() # Convert Y to numeric array
clusters = train_set[ID_VAR].to_numpy() # Get ID variables
z = np.ones((train_set.shape[0], 1)) # Create random effects matrix with ones

best_score = float('inf')
best_params = {}
results = []  # Initialize a list to store the results of each iteration

# Loop through all possible combinations of parameters
total_combinations = len(list(itertools.product(*param_grid.values())))  # Calculate total combinations
for idx, params in enumerate(itertools.product(*param_grid.values())):
    n_estimators, max_depth, min_samples_split, max_iter, n_splits = params
    
    # Print progress
    progress_percentage = (idx + 1) / total_combinations * 100
    print(f"Combination: {params}\n")
    print(f"Progress: {progress_percentage:.2f}% completed\n")
    scores, prev, ptev, oob_scores = [], [], [], []  # Initialize lists for scores

    # K-fold cross-validation with variable n_splits
    kf = KFold(n_splits=n_splits)
    for train_index, test_index in kf.split(X):
        print("Train indices:", train_index)
        print("Test indices:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Use .iloc for row selection
        y_train, y_test = y[train_index], y[test_index]
        clusters_train, clusters_test = clusters[train_index], pd.Series(clusters[test_index])
        z_train, z_test = z[train_index], z[test_index]

        print("Length of clusters_train:", len(clusters_train))
        print("Length of clusters_test:", len(clusters_test))
        print("Train indices:", train_index)
        print("Test indices:", test_index)

        model = MERF(
                # Specify the fixed effects model as a Random Forest Regressor
            fixed_effects_model=RandomForestRegressor(
                n_estimators=n_estimators,  # Number of trees in the forest
                max_depth=max_depth,  # Maximum depth of each tree
                min_samples_split=min_samples_split,  # Minimum samples required to split an internal node
                n_jobs=1,  # Number of jobs to run in parallel
                oob_score=True  # Whether to use out-of-bag samples to estimate the R^2 on unseen data
                ),
                # Generalized Linear Model (GLM) early stopping threshold
            gll_early_stop_threshold=None,  # No early stopping threshold set
                # Maximum number of iterations for the MERF algorithm
            max_iterations=max_iter  # Maximum number of iterations to run the MERF algorithm
            )
        model.fit(X_train.select_dtypes(include=[np.number]), z_train, pd.Series(clusters_train), y_train)
        y_pred = model.predict(X_test, z_test, clusters_test)
        scores.append(np.mean((y_pred - y_test) ** 2)) # MSE
        
        # Calculate ptev and prev
        total_variance = np.var(y_test)
        random_effect_variance = np.var(y_test - y_pred)
        fixed_effect_variance = total_variance - random_effect_variance

        ptev.append(np.mean(fixed_effect_variance / total_variance if total_variance > 0 else 0))
        prev.append(np.mean(random_effect_variance / total_variance if total_variance > 0 else 0))

        # Calculate OOB score
        forest = model.trained_fe_model
        oob_score = round(forest.oob_score_ * 100, 1)  # percent variation
        oob_scores.append(oob_score)

        # Print ptev, prev, and OOB score for the current iteration
        print(f"Combination, ptev: {np.mean(ptev):.4f}, prev: {np.mean(prev):.4f}, OOB Score: {oob_score:.4f}")

    # Calculate the mean of the scores for the current combination of parameters
    mean_score = np.mean(scores)
    if mean_score < best_score:
        best_score = mean_score
        best_params = params

    # Append the results of the current iteration to the results list
    result_dict = {
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'max_iter': max_iter,
        'n_splits': n_splits,
        'mean_mse_score': mean_score,
        'mean_prev': np.mean(prev),
        'mean_ptev': np.mean(ptev),
        'oob_score': np.mean(oob_scores)
    }
    results.append(result_dict)

print("Best parameters:", best_params)
print("Best score:", best_score)

# Convert the results list to a DataFrame and save it to a CSV file
results_df = pd.DataFrame(results)
results_df.to_csv(OUT, index=False)

print("---------- Run MERF models ----------")
r2_values, results_df = run_merf_analysis2(
    X, Y, Z, clusters,
    X_new, Y_new, Z_new, clusters_new,
    results_df,
    output_dir, r2_out, r2_adj_out, feature_imp_out, results_filename, time_new)

# Print the R-squared values and the results DataFrame
print("R-squared values:", r2_values)
print("Results DataFrame:\n", results_df)

Subset for basic created with shape: (355, 5)
Subset for meta_keep created with shape: (355, 14)
Subset for only_grs created with shape: (355, 4)
Subset for only_taxa created with shape: (355, 182)
Subset for only_micom created with shape: (355, 94)
Subset for only_pathway created with shape: (355, 270)
---------- Select predictors for training set ----------
Length of X: 255, Length of clusters_train: 255, Length of Y: 255
Final columns after drop: ['time', 'N10.formyl.tetrahydrofolate.biosynthesis', 'homolactic.fermentation', 'glycolysis.III..from.glucose.', 'superpathway.of.arginine.and.polyamine.biosynthesis', 'arginine..ornithine.and.proline.interconversion', 'L.arginine.biosynthesis.I..via.L.ornithine.', 'L.arginine.biosynthesis.II..acetyl.cycle.', 'chorismate.biosynthesis.I', 'superpathway.of.L.aspartate.and.L.asparagine.biosynthesis', 'biotin.biosynthesis.I', 'superpathway.of.branched.amino.acid.biosynthesis', 'Calvin.Benson.Bassham.cycle', 'pyruvate.fermentation.to.butanoate',

INFO     [merf.py:307] Training GLL is 303.8209884949022 at iteration 1.
INFO     [merf.py:307] Training GLL is 377.0656741409432 at iteration 2.


Combination, ptev: 0.1583, prev: 0.8417, OOB Score: 27.9000
Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84 170 171 172 173 174
 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192
 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210
 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228
 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246
 247 248 249 250 251 252 253 254]
Test indices: [ 85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102
 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 

INFO     [merf.py:307] Training GLL is 386.6580675602936 at iteration 1.
INFO     [merf.py:307] Training GLL is 478.21749487281494 at iteration 2.


Combination, ptev: 0.3624, prev: 0.6376, OOB Score: 13.5000
Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169]
Test indices: [170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223
 

...

INFO     [merf.py:307] Training GLL is 737.6548474164294 at iteration 1.
INFO     [merf.py:307] Training GLL is 744.0825325150547 at iteration 2.
INFO     [merf.py:307] Training GLL is 705.7660615424998 at iteration 3.
INFO     [merf.py:307] Training GLL is 681.4334663448976 at iteration 4.
INFO     [merf.py:307] Training GLL is 663.2864778499666 at iteration 5.
INFO     [merf.py:307] Training GLL is 646.0070947724013 at iteration 6.
INFO     [merf.py:307] Training GLL is 634.90830810383 at iteration 7.
INFO     [merf.py:307] Training GLL is 625.1766346953709 at iteration 8.
INFO     [merf.py:307] Training GLL is 617.6021330262378 at iteration 9.
INFO     [merf.py:307] Training GLL is 610.3282306904974 at iteration 10.


Combination, ptev: 0.6862, prev: 0.3138, OOB Score: 58.4000
Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 

INFO     [merf.py:307] Training GLL is 758.0247664520482 at iteration 1.
INFO     [merf.py:307] Training GLL is 778.716945762574 at iteration 2.
INFO     [merf.py:307] Training GLL is 732.2079410643136 at iteration 3.
INFO     [merf.py:307] Training GLL is 698.7278761429467 at iteration 4.
INFO     [merf.py:307] Training GLL is 671.8275006461403 at iteration 5.
INFO     [merf.py:307] Training GLL is 651.3497804777506 at iteration 6.
INFO     [merf.py:307] Training GLL is 640.1887077439658 at iteration 7.
INFO     [merf.py:307] Training GLL is 631.9693252581004 at iteration 8.
INFO     [merf.py:307] Training GLL is 628.1235662627478 at iteration 9.
INFO     [merf.py:307] Training GLL is 626.6438304457006 at iteration 10.


Combination, ptev: 0.7049, prev: 0.2951, OOB Score: 55.4000
Best parameters: (1000, None, 0.1, 10, 10)
Best score: 6.08461480241211
---------- Run MERF models ----------


TypeError: run_merf_analysis2() takes 14 positional arguments but 15 were given